In [1]:
import numpy as np
import json
import glob
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [37]:
data = pd.read_csv('output.csv',encoding='latin1')
#data = pd.read_excel('Pranav_RGA_Reports.xlsx',usecols=['Description'])
print(data)

There was an error parsing 'Restaurant_Reviews.tsv'.


In [24]:
from nltk.corpus import stopwords

stopwords = stopwords.words("english")
# stopwords.append('application')
# stopwords.append('inspection')
# stopwords.append('review')
# stopwords.append('customer')

print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [29]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags and token.text.lower() not in stopwords:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts = lemmatization(data['review'].tolist())

In [30]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['look', 'spit', 'food', 'normal', 'transaction', 'chill', 'polite', 'want', 'eat', 'try', 'think', 'milky', 'white', 'clear', 'substance', 'food', 'sure', 'come', 'back']


In [31]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['look', 'spit', 'food', 'normal', 'transaction', 'chill', 'polite', 'want', 'eat', 'try', 'think', 'milky', 'white', 'clear', 'substance', 'food', 'sure', 'come', 'back']


In [32]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [33]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [34]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [35]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.251074 -0.292911       1        1  51.335920
2      0.287719  0.216647       2        1  17.682118
0     -0.217547  0.318767       3        1  16.421745
3     -0.321247 -0.242503       4        1  14.560218, topic_info=             Term          Freq         Total Category  logprob  loglift
63             ½i  48539.000000  48539.000000  Default  30.0000  30.0000
62        service   7269.000000   7269.000000  Default  29.0000  29.0000
5            food   6717.000000   6717.000000  Default  28.0000  28.0000
160          good   5228.000000   5228.000000  Default  27.0000  27.0000
29          place   3714.000000   3714.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
2921  deteriorate     23.249968     25.123790   Topic4  -7.7617   1.8494
5750          hav     13.419908     14.467679   Topic4  -8.3113   1.8517
9375           re     20.485019     22.305574   Topic4  -7.8884   1.8417
9485          doi     21.306070     26.204583   Topic4  -7.8491   1.7199
1984           th     30.820736    126.406976   Topic4  -7.4799   0.5156

[168 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
9311      4  0.968624     actu
19        3  0.999714   always
1088      3  0.995330  amazing
822       2  0.999168     area
95        1  0.999434      ask
...     ...       ...      ...
3741      4  0.950881      ½re
382       4  0.998458       ½s
423       4  0.998165       ½t
764       4  0.994968      ½ve
7355      4  0.916197     ½wei

[157 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4])